In [8]:

from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [9]:
#FUNCTION TO LOAD ANY TYPE OF DATASET
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
        return dataset

In [10]:
#CONVERT STRING COLMN TO FLOAT
def str_colmn_to_float(dataset,column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [11]:
#find the min and max values for each colmn
def dataset_minmax(dataset):
    minmax = list()
    for i in range (len(dataset[0])):
        col_values = [row[i]for row in dataset]
        value_min = min (col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

In [12]:
#rescale dataset column to range 0 -1
def normalize_dataset(dataset, minimax):
    for row in dataset:
        for i in range (len(row)):
            row[i] = (row[i] - minimax[i][0]/ minimax[i][1] - minimax[i][0])

In [13]:
#
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset)/n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
        return dataset_split

In [14]:
#calculate root mean square error
def rmse_metric(actual,predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error **2)
        mean_error = sum_error / float(len(actual))
        return sqrt(mean_error)

In [15]:
#evaluate on algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds =  cross_validation_split(dataset,n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        rmse = rmse_metric(actual, predicted)
        scores.append(rmse)
    return scores

In [16]:
#make a prediction with coefficient
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
        return yhat

In [17]:
#linear regression algorithm with stochastic gradient descent
def linear_regression_sgd(train, test, l_rate, n_epach):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epach)
    for row in test:
        yhat = predict(row,coef)
        predictions.append(yhat)

    return(predictions)
        

In [18]:
dataset = [[1,1], [2,3], [4,3],[3,2],[5,5]]
coef = [0,4,0,8]

for row in dataset:
    yhat = predict(row,coef)
    print("Expected=%.3f, predicted=%.3f" %(row[-1],yhat))

Expected=1.000, predicted=4.000
Expected=3.000, predicted=8.000
Expected=3.000, predicted=16.000
Expected=2.000, predicted=12.000
Expected=5.000, predicted=20.000


In [19]:
#Estimate Linear regression
def coefficient_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error =+ error**2
            coef[0] = coef[0] - l_rate * error
            for i in range (len(row)-1):
                coef[0] = coef[i+1] - l_rate * error * row[i]
        print('>epoch=%d, lrate=%.3f, error=%.3f' %(epoch, l_rate, sum_error))
    return coef
    

In [20]:
dataset = [[1,1],[2,3],[4,3],[3,2],[5,5]]
l_rate = 0.001
n_epoch = 10
coef = coefficient_sgd(dataset, l_rate, n_epoch)
print(coef)

>epoch=0, lrate=0.001, error=24.940
>epoch=1, lrate=0.001, error=24.940
>epoch=2, lrate=0.001, error=24.940
>epoch=3, lrate=0.001, error=24.940
>epoch=4, lrate=0.001, error=24.940
>epoch=5, lrate=0.001, error=24.940
>epoch=6, lrate=0.001, error=24.940
>epoch=7, lrate=0.001, error=24.940
>epoch=8, lrate=0.001, error=24.940
>epoch=9, lrate=0.001, error=24.940
[0.024970179640117003, 0.0]


In [30]:
#load data and prepare data
filename = 'C:\\Users\\cmupenzi\\Desktop\\AUCA\Machine Learning\\winequality-white.csv'
dataset = load_csv(filename)
dataset
dataset_minmax(dataset)

#string to float
for i in range(len(dataset[0])):
    str_colmn_to_float(dataset, i)
    minmax = dataset_minmax(dataset, minmax)
normalize_dataset(dataset, minmax)

#evaluate algo

n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate,n_epoch)
print('Scores: %s' % scores)
print('mean RMSE: %.3f' % (sum(scores)/float(len(scores))))


TypeError: dataset_minmax() takes 1 positional argument but 2 were given